# 🚀 Notebook 01 : Simulation de Flux Temps Réel

**Auteur** : Oumaro Titans DJIGUIMDE  
**Date** : Février 2026  
**Objectif** : Simuler un flux de transactions financières en temps réel

---

## 🎯 Objectifs de ce notebook

1. Comprendre la génération de flux de données
2. Simuler des transactions réalistes
3. Visualiser le flux généré
4. Valider la qualité des données
5. Préparer le pipeline d'ingestion

## 📦 Importation des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import random

# Configuration
pd.set_option('display.max_columns', None)
np.random.seed(42)
random.seed(42)

# Style visualisation
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

print("✅ Bibliothèques importées avec succès")

## 1️⃣ Configuration du générateur de transactions

In [ ]:
print("="*70)
print("⚙️  CONFIGURATION DU GÉNÉRATEUR")
print("="*70)

# Villes sénégalaises
VILLES = ['Dakar', 'Thiès', 'Saint-Louis', 'Kaolack', 'Ziguinchor']

# Types de transactions avec pondération
TRANSACTION_TYPES = {
    'transfer': 0.45,      # 45% transferts P2P
    'payment': 0.30,       # 30% paiements marchands
    'withdrawal': 0.20,    # 20% retraits
    'deposit': 0.05        # 5% dépôts
}

# Opérateurs Mobile Money
OPERATEURS = ['Orange Money', 'Wave', 'Free Money', 'Wizall']

# Plages de montants par type (FCFA)
MONTANT_RANGES = {
    'transfer': (1000, 100000),
    'payment': (500, 50000),
    'withdrawal': (5000, 200000),
    'deposit': (10000, 500000)
}

print(f"\n📍 Villes : {', '.join(VILLES)}")
print(f"📱 Opérateurs : {', '.join(OPERATEURS)}")
print(f"💳 Types de transactions : {list(TRANSACTION_TYPES.keys())}")
print("="*70)

## 2️⃣ Fonctions de génération

In [ ]:
def generer_montant(transaction_type):
    """
    Génère un montant réaliste selon le type de transaction
    Utilise une distribution log-normale pour plus de réalisme
    """
    min_val, max_val = MONTANT_RANGES[transaction_type]
    
    montant = np.random.lognormal(
        mean=np.log(min_val + (max_val - min_val) / 3),
        sigma=0.8
    )
    
    return max(min_val, min(max_val, round(montant, 0)))

def generer_user_id():
    """
    Génère un user_id au format téléphone sénégalais
    Format: user_7XXXXXXXX
    """
    prefixes = ['77', '78', '76', '70', '75']  # Préfixes sénégalais
    prefix = random.choice(prefixes)
    numero = f"{prefix}{random.randint(1000000, 9999999)}"
    return f"user_{numero}"

def generer_transaction():
    """
    Génère une transaction complète
    """
    # Type
    trans_type = np.random.choice(
        list(TRANSACTION_TYPES.keys()),
        p=list(TRANSACTION_TYPES.values())
    )
    
    return {
        'transaction_id': f"TXN{int(time.time()*1000)}_{random.randint(1000,9999)}",
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'user_id': generer_user_id(),
        'amount': generer_montant(trans_type),
        'city': random.choice(VILLES),
        'transaction_type': trans_type,
        'operator': random.choice(OPERATEURS),
        'status': 'completed'
    }

print("✅ Fonctions de génération définies")

## 3️⃣ Test de génération unitaire

In [ ]:
print("🧪 TEST : Génération de 5 transactions")
print("="*70)

for i in range(5):
    trans = generer_transaction()
    print(f"\nTransaction {i+1} :")
    for key, value in trans.items():
        print(f"   {key:20s} : {value}")

print("\n" + "="*70)

## 4️⃣ Génération d'un batch de transactions

In [ ]:
def generer_batch(n_transactions=10000, periode_jours=7):
    """
    Génère un batch de transactions sur une période donnée
    """
    print(f"🔄 Génération de {n_transactions:,} transactions sur {periode_jours} jours...")
    
    transactions = []
    base_date = datetime.now() - timedelta(days=periode_jours)
    
    for i in range(n_transactions):
        if (i + 1) % 2000 == 0:
            print(f"   ✓ {i+1:,} / {n_transactions:,}")
        
        trans = generer_transaction()
        
        # Timestamp aléatoire dans la période
        offset_hours = random.randint(0, periode_jours * 24)
        timestamp = base_date + timedelta(hours=offset_hours)
        trans['timestamp'] = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        
        transactions.append(trans)
    
    df = pd.DataFrame(transactions)
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    print(f"\n✅ {len(df):,} transactions générées")
    return df

# Génération
df_stream = generer_batch(n_transactions=10000, periode_jours=7)

## 5️⃣ Aperçu du flux généré

In [ ]:
print("👀 APERÇU DU FLUX")
print("="*70)
df_stream.head(10)

In [ ]:
# Informations
print("\n📋 Informations :")
df_stream.info()

## 6️⃣ Statistiques du flux

In [ ]:
print("="*70)
print("📊 STATISTIQUES DU FLUX GÉNÉRÉ")
print("="*70)

print(f"\n📌 Volume :")
print(f"   • Transactions : {len(df_stream):,}")
print(f"   • Période : {df_stream['timestamp'].min()} → {df_stream['timestamp'].max()}")

print(f"\n💰 Montants :")
print(f"   • Total : {df_stream['amount'].sum():,.0f} FCFA")
print(f"   • Moyenne : {df_stream['amount'].mean():,.0f} FCFA")
print(f"   • Médiane : {df_stream['amount'].median():,.0f} FCFA")
print(f"   • Min : {df_stream['amount'].min():,.0f} FCFA")
print(f"   • Max : {df_stream['amount'].max():,.0f} FCFA")

print(f"\n🏙️  Par ville :")
for city, count in df_stream['city'].value_counts().items():
    pct = (count / len(df_stream)) * 100
    print(f"   • {city:15s} : {count:5,} ({pct:5.1f}%)")

print(f"\n💳 Par type :")
for ttype, count in df_stream['transaction_type'].value_counts().items():
    pct = (count / len(df_stream)) * 100
    print(f"   • {ttype:15s} : {count:5,} ({pct:5.1f}%)")

print(f"\n📱 Par opérateur :")
for op, count in df_stream['operator'].value_counts().items():
    pct = (count / len(df_stream)) * 100
    print(f"   • {op:20s} : {count:5,} ({pct:5.1f}%)")

print("\n" + "="*70)

## 7️⃣ Visualisations du flux

In [ ]:
# Distribution des montants
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogramme
axes[0].hist(df_stream['amount'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution des Montants', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Montant (FCFA)')
axes[0].set_ylabel('Fréquence')
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(df_stream['amount'], vert=True)
axes[1].set_title('Boxplot des Montants', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Montant (FCFA)')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Répartition par ville et type
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Par ville
df_stream['city'].value_counts().plot(kind='bar', ax=axes[0], color='coral', alpha=0.8)
axes[0].set_title('Transactions par Ville', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Ville')
axes[0].set_ylabel('Nombre de transactions')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# Par type
df_stream['transaction_type'].value_counts().plot(kind='bar', ax=axes[1], color='lightgreen', alpha=0.8)
axes[1].set_title('Transactions par Type', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Type')
axes[1].set_ylabel('Nombre de transactions')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Distribution par opérateur (Pie chart)
plt.figure(figsize=(10, 8))
df_stream['operator'].value_counts().plot(kind='pie', autopct='%1.1f%%', 
                                           startangle=90, colors=sns.color_palette('Set3'))
plt.title('Répartition par Opérateur Mobile Money', fontsize=16, fontweight='bold')
plt.ylabel('')
plt.tight_layout()
plt.show()

## 8️⃣ Validation de la qualité

In [ ]:
print("🔍 VALIDATION DE LA QUALITÉ")
print("="*70)

# Valeurs manquantes
missing = df_stream.isnull().sum()
print(f"\n1️⃣ Valeurs manquantes :")
if missing.sum() == 0:
    print("   ✅ Aucune valeur manquante")
else:
    print(missing[missing > 0])

# Doublons
doublons = df_stream.duplicated().sum()
print(f"\n2️⃣ Doublons :")
print(f"   {'✅' if doublons == 0 else '⚠️'} {doublons} doublons détectés")

# Cohérence des montants
print(f"\n3️⃣ Cohérence des montants :")
for ttype, (min_val, max_val) in MONTANT_RANGES.items():
    df_type = df_stream[df_stream['transaction_type'] == ttype]
    hors_range = ((df_type['amount'] < min_val) | (df_type['amount'] > max_val)).sum()
    print(f"   • {ttype:15s} : {hors_range} hors range")

# Unicité des IDs
unique_ids = df_stream['transaction_id'].nunique()
print(f"\n4️⃣ Unicité des IDs :")
print(f"   {'✅' if unique_ids == len(df_stream) else '⚠️'} {unique_ids:,} / {len(df_stream):,} IDs uniques")

print("\n" + "="*70)
print("✅ VALIDATION RÉUSSIE : Données prêtes pour le pipeline")
print("="*70)

## 9️⃣ Sauvegarde du flux

In [ ]:
# Sauvegarde
output_path = '../data/transactions_stream.csv'
df_stream.to_csv(output_path, index=False, encoding='utf-8')

print("="*70)
print("💾 SAUVEGARDE DU FLUX")
print("="*70)
print(f"\n✅ Fichier sauvegardé : {output_path}")
print(f"📊 Taille : {df_stream.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"📦 Lignes : {len(df_stream):,}")
print(f"📋 Colonnes : {len(df_stream.columns)}")
print("\n" + "="*70)

## 📌 Résumé

### ✅ Ce qui a été fait

1. **Configuration** du générateur de transactions
2. **Implémentation** des fonctions de génération
3. **Génération** de 10 000 transactions sur 7 jours
4. **Validation** de la qualité des données
5. **Visualisation** des distributions
6. **Sauvegarde** du flux pour le pipeline

### 📊 Caractéristiques du flux

- ✅ 10 000 transactions
- ✅ 5 villes sénégalaises
- ✅ 4 opérateurs Mobile Money
- ✅ 4 types de transactions
- ✅ Distribution réaliste des montants
- ✅ Timestamps sur 7 jours

### 🚀 Prochaine étape

**Notebook 02** : Transformation des données
- Parsing des timestamps
- Extraction de features temporelles
- Enrichissement des données
- Agrégations
- Calcul d'indicateurs